In [1]:
import numpy as np

In [2]:
with open('movie-xids.npy', 'rb') as f:
    xids = np.load(f, allow_pickle=True)
    
with open('movie-xmasks.npy', 'rb') as f:
    xmasks = np.load(f, allow_pickle=True)
    
with open('movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle=True)

In [3]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0], 'GPU')

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((xids, xmasks, labels))
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [6]:
#tensorflow needs (input, output). Here we have two input tensors and one output tensor
#mapping that

def map_func(ids, masks, labels):
    return {'input_ids': ids,
            'attention_masks': masks}, labels

In [7]:
dataset = dataset.map(map_func)

In [8]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_masks': TensorSpec(shape=(512,), dtype=tf.int64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [10]:
batch_size = 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [11]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_masks': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [12]:
split = 0.9
size = int((xids.shape[0] / batch_size) * split)
size

8778

In [13]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [17]:
tf.data.Dataset.save(train_ds, 'train')
tf.data.Dataset.save(val_ds, 'val')

In [18]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_masks': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [19]:
train_ds.element_spec == val_ds.element_spec

True